In [ ]:
%matplotlib qt5
import os
import glob
import hyperspy.api as hs
import pyxem as pxm
import matplotlib.pyplot as plt
import h5py as h5
import dask.array as da

import numpy as np

In [ ]:
os.chdir(r"D:\dg606\SED\Uncalibrated_au_x")

In [7]:
path = r"20221130_141255.hspy"
with h5.File(path) as f:
    
    data = f['Experiments/__unnamed__/data']
    
    chunks = (30,30,515,515)
    x = da.from_array(data, chunks=chunks)
    s = hs.signals.Signal2D(x).as_lazy()
    s.compute()


[########################################] | 100% Completed | 38.6s


In [8]:
s.plot()

In [11]:
summed = s.sum()

In [13]:
summed.plot(norm='log')

In [ ]:
dirc_list = glob.glob('*.hspy')

In [ ]:
aff_trans = np.load(r"D:\dg606\SED\aff_trans.npy")
for dirc in dirc_list:
    print(dirc)
    os.chdir(r'D:\dg606\SED\Uncalibrated_au_x')
    f = h5.File(dirc)           # Load the file
    data = f['Experiments/__unnamed__/data']                 # Get the data
    chunks = (40,40,515,515)                     # Chunk as appropriate
    x = da.from_array(data, chunks=chunks) # Wrap the data in dask
    #s = hs.signals.Signal2D(x).as_lazy() # Create the lazy signal
    s= pxm.signals.ElectronDiffraction2D(x).as_lazy() #this is needed because of a bug in the code 
    s.compute()
    s.center_direct_beam(method='blur', sigma = 1)
    s.apply_affine_transformation(aff_trans,keep_dtype=True, inplace = True)
    

 
    os.chdir(r'D:\dg606\SED\Centered_and_aff_trans_au_x')
    s.save(dirc[:-5]+'.hdf5',compression='gzip')
    summed = s.T.sum().data
    np.save(dirc[:-5]+'_spi.npy',summed)
    gc.collect()

20221130_141255.hspy
[########################################] | 100% Completed | 50.8s


Overwrite 'D:\dg606\SED\Centered_and_aff_trans_au_x\20221130_141255.hdf5' (y/n)?
 y


NameError: name 'gc' is not defined